In [4]:
# ============================================================
# 🌌 OMEGA INFINITY v4 - COMPLETE COMPRESSION TOOL
# ============================================================
# Meta-Recursive Markov Chain Compression
# f(m) = m applied recursively: f(f(f(...f(m)...)))
# ============================================================

import numpy as np
from collections import defaultdict
import struct
import zlib
import base64
import os
import json
from datetime import datetime
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ============================================================
# CORE COMPRESSION ENGINE
# ============================================================

class MetaMarkovCompressor:
    """Base adaptive Markov compressor"""

    def __init__(self, order=3):
        self.order = order

    def compress_to_ranks(self, text):
        """Compress text to list of ranks + metadata"""
        if len(text) <= self.order:
            return {
                'prefix': text,
                'ranks': [],
                'max_values': [],
                'alphabet': list(set(text)) if text else []
            }

        transitions = defaultdict(lambda: defaultdict(int))
        alphabet = sorted(set(text))
        char_to_idx = {c: i for i, c in enumerate(alphabet)}

        prefix = text[:self.order]
        ranks = []
        max_values = []

        for i in range(self.order, len(text)):
            context = text[i-self.order:i]
            current_char = text[i]

            if context in transitions:
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                if current_char in seen_list:
                    rank = seen_list.index(current_char)
                    max_val = len(seen_list) + 1
                else:
                    rank = len(seen_list)
                    max_val = len(seen_list) + 1

                ranks.append(rank)
                max_values.append(max_val)

                if current_char not in seen_list:
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = unseen.index(current_char)
                    ranks.append(unseen_idx)
                    max_values.append(len(unseen))
            else:
                rank = char_to_idx[current_char]
                ranks.append(rank)
                max_values.append(len(alphabet))

            transitions[context][current_char] += 1

        return {
            'prefix': prefix,
            'ranks': ranks,
            'max_values': max_values,
            'alphabet': alphabet,
            'original_len': len(text)
        }

    def decompress_from_ranks(self, data):
        """Reconstruct text from ranks"""
        prefix = data['prefix']
        ranks = data['ranks']
        max_values = data['max_values']
        alphabet = data['alphabet']
        original_len = data['original_len']

        if not ranks:
            return prefix

        result = list(prefix)
        transitions = defaultdict(lambda: defaultdict(int))

        rank_idx = 0

        while len(result) < original_len and rank_idx < len(ranks):
            context = ''.join(result[-self.order:])

            if context in transitions:
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                rank = ranks[rank_idx]
                rank_idx += 1

                if rank < len(seen_list):
                    char = seen_list[rank]
                else:
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = ranks[rank_idx]
                    rank_idx += 1
                    char = unseen[unseen_idx]
            else:
                rank = ranks[rank_idx]
                rank_idx += 1
                char = alphabet[rank]

            result.append(char)
            transitions[context][char] += 1

        return ''.join(result)


class OmegaInfinityCompressor:
    """
    Omega Infinity v4 - Meta-Recursive Compression
    Applies Markov compression recursively: f(f(f(m)))
    """

    VERSION = "4.0"
    MAGIC_BYTES = b'OMEGA'

    def __init__(self, base_order=4, meta_levels=3, verbose=True):
        self.base_order = base_order
        self.meta_levels = meta_levels
        self.verbose = verbose

    def log(self, message):
        """Print message if verbose mode is on"""
        if self.verbose:
            print(message)

    def compress(self, data, is_binary=False):
        """
        Compress data (text or binary)

        Args:
            data: String or bytes to compress
            is_binary: True if data is binary

        Returns:
            Compressed bytes
        """
        self.log(f"{'='*60}")
        self.log(f"🌌 OMEGA INFINITY v{self.VERSION} - COMPRESSION")
        self.log(f"{'='*60}")

        # Convert to text representation if binary
        if is_binary or isinstance(data, bytes):
            text = base64.b64encode(data if isinstance(data, bytes) else data.encode()).decode('ascii')
            data_type = 'binary'
        else:
            text = data
            data_type = 'text'

        original_size = len(text.encode('utf-8'))
        self.log(f"📝 Original size: {original_size:,} bytes")
        self.log(f"📊 Base order: {self.base_order}")
        self.log(f"🔄 Meta-levels: {self.meta_levels}")
        self.log(f"📁 Data type: {data_type}")
        self.log(f"{'='*60}\n")

        # Store metadata for each level
        levels_data = []
        current_text = text

        for level in range(self.meta_levels):
            self.log(f"⚙️  Level {level + 1}:")
            self.log(f"   📥 Input: {len(current_text):,} symbols")

            # Adjust order based on level
            order = max(1, self.base_order - level)
            compressor = MetaMarkovCompressor(order=order)

            # Compress to ranks
            data_compressed = compressor.compress_to_ranks(current_text)

            self.log(f"   📦 Ranks generated: {len(data_compressed['ranks']):,}")
            self.log(f"   📉 Max rank: {max(data_compressed['ranks']) if data_compressed['ranks'] else 0}")
            self.log(f"   🔤 Alphabet: {len(data_compressed['alphabet'])} symbols")

            # Save metadata for this level
            level_meta = {
                'order': order,
                'prefix': data_compressed['prefix'],
                'alphabet': data_compressed['alphabet'],
                'max_values': data_compressed['max_values'],
                'original_len': data_compressed['original_len']
            }
            levels_data.append(level_meta)

            # Convert ranks to string for next level
            if data_compressed['ranks']:
                max_rank = max(data_compressed['ranks'])
                if max_rank < 256:
                    current_text = ''.join(chr(r) for r in data_compressed['ranks'])
                else:
                    current_text = ','.join(map(str, data_compressed['ranks']))
            else:
                current_text = ""
                break

            self.log(f"   📤 Output: {len(current_text):,} symbols")

            # Stop if already very small
            if len(current_text) < 10:
                self.log(f"   ⏹️  Small enough, stopping.")
                break

            self.log("")

        # Final ranks
        final_ranks = [ord(c) for c in current_text] if current_text else []

        # Pack everything
        package = self._pack_all(levels_data, final_ranks, data_type, original_size)
        compressed = zlib.compress(package, level=9)

        # Add magic header
        header = self.MAGIC_BYTES + struct.pack('B', int(float(self.VERSION)))
        final_compressed = header + compressed

        compressed_size = len(final_compressed)
        ratio = (1 - compressed_size / original_size) * 100

        self.log(f"\n{'='*60}")
        self.log(f"📊 FINAL RESULTS")
        self.log(f"{'='*60}")
        self.log(f"📁 Original: {original_size:,} bytes")
        self.log(f"📦 Compressed: {compressed_size:,} bytes")
        self.log(f"📉 Ratio: {ratio:.2f}%")
        self.log(f"🔢 Factor: {original_size/compressed_size:.2f}x")
        self.log(f"🔄 Levels used: {len(levels_data)}")
        self.log(f"{'='*60}")

        return final_compressed

    def decompress(self, compressed_data):
        """
        Decompress data

        Args:
            compressed_data: Compressed bytes

        Returns:
            Original data (string or bytes)
        """
        self.log(f"\n{'='*60}")
        self.log(f"🔓 OMEGA INFINITY - DECOMPRESSION")
        self.log(f"{'='*60}")

        # Check magic header
        if not compressed_data.startswith(self.MAGIC_BYTES):
            raise ValueError("Invalid Omega Infinity file format!")

        # Extract version and data
        version = struct.unpack('B', compressed_data[5:6])[0]
        self.log(f"📋 File version: {version}")

        # Decompress
        data = zlib.decompress(compressed_data[6:])
        pos = 0

        # Read data type
        data_type_len = struct.unpack('B', data[pos:pos+1])[0]; pos += 1
        data_type = data[pos:pos+data_type_len].decode('utf-8'); pos += data_type_len

        # Read original size
        original_size = struct.unpack('I', data[pos:pos+4])[0]; pos += 4

        # Number of levels
        num_levels = data[pos]; pos += 1
        self.log(f"🔄 Levels to process: {num_levels}")
        self.log(f"📁 Data type: {data_type}")

        # Read each level
        levels_data = []
        for i in range(num_levels):
            level = {}

            level['order'] = data[pos]; pos += 1
            level['original_len'] = struct.unpack('I', data[pos:pos+4])[0]; pos += 4

            prefix_len = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            level['prefix'] = data[pos:pos+prefix_len].decode('utf-8'); pos += prefix_len

            alphabet_len = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            level['alphabet'] = list(data[pos:pos+alphabet_len].decode('utf-8')); pos += alphabet_len

            num_max_values = struct.unpack('I', data[pos:pos+4])[0]; pos += 4
            level['max_values'] = []
            for _ in range(num_max_values):
                mv = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
                level['max_values'].append(mv)

            levels_data.append(level)

        # Final ranks
        num_final = struct.unpack('I', data[pos:pos+4])[0]; pos += 4
        final_ranks = []
        for _ in range(num_final):
            r = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            final_ranks.append(r)

        # Reconstruct from last level to first
        current_ranks = final_ranks

        for i in range(num_levels - 1, -1, -1):
            level = levels_data[i]
            self.log(f"   ⬆️  Level {i + 1}: {len(current_ranks):,} ranks → ", end="")

            decomp_data = {
                'prefix': level['prefix'],
                'ranks': current_ranks,
                'max_values': level['max_values'],
                'alphabet': level['alphabet'],
                'original_len': level['original_len']
            }

            compressor = MetaMarkovCompressor(order=level['order'])
            text = compressor.decompress_from_ranks(decomp_data)

            self.log(f"{len(text):,} symbols")

            if i > 0:
                current_ranks = [ord(c) for c in text]
            else:
                result = text

        # Convert back from base64 if binary
        if data_type == 'binary':
            result = base64.b64decode(result)

        final_size = len(result) if isinstance(result, bytes) else len(result.encode('utf-8'))
        self.log(f"\n✅ Recovered: {final_size:,} bytes")
        self.log(f"{'='*60}")

        return result

    def _pack_all(self, levels_data, final_ranks, data_type, original_size):
        """Pack all levels into bytes"""
        package = bytearray()

        # Data type
        data_type_bytes = data_type.encode('utf-8')
        package.append(len(data_type_bytes))
        package.extend(data_type_bytes)

        # Original size
        package.extend(struct.pack('I', original_size))

        # Number of levels
        package.append(len(levels_data))

        # Each level
        for level in levels_data:
            package.append(level['order'])
            package.extend(struct.pack('I', level['original_len']))

            prefix_bytes = level['prefix'].encode('utf-8')
            package.extend(struct.pack('H', len(prefix_bytes)))
            package.extend(prefix_bytes)

            alphabet_str = ''.join(level['alphabet'])
            alphabet_bytes = alphabet_str.encode('utf-8')
            package.extend(struct.pack('H', len(alphabet_bytes)))
            package.extend(alphabet_bytes)

            package.extend(struct.pack('I', len(level['max_values'])))
            for mv in level['max_values']:
                package.extend(struct.pack('H', min(mv, 65535)))

        # Final ranks
        package.extend(struct.pack('I', len(final_ranks)))
        for r in final_ranks:
            package.extend(struct.pack('H', r))

        return bytes(package)

    def compress_file(self, input_path, output_path=None):
        """
        Compress a file

        Args:
            input_path: Path to input file
            output_path: Path to output file (optional)

        Returns:
            Path to compressed file
        """
        if output_path is None:
            output_path = input_path + '.omega'

        # Read file
        with open(input_path, 'rb') as f:
            data = f.read()

        # Compress
        compressed = self.compress(data, is_binary=True)

        # Write compressed file
        with open(output_path, 'wb') as f:
            f.write(compressed)

        return output_path

    def decompress_file(self, input_path, output_path=None):
        """
        Decompress a file

        Args:
            input_path: Path to compressed file
            output_path: Path to output file (optional)

        Returns:
            Path to decompressed file
        """
        if output_path is None:
            if input_path.endswith('.omega'):
                output_path = input_path[:-6]
            else:
                output_path = input_path + '.decompressed'

        # Read compressed file
        with open(input_path, 'rb') as f:
            compressed = f.read()

        # Decompress
        data = self.decompress(compressed)

        # Write decompressed file
        mode = 'wb' if isinstance(data, bytes) else 'w'
        with open(output_path, mode) as f:
            f.write(data)

        return output_path


# ============================================================
# INTERACTIVE USER INTERFACE
# ============================================================

class OmegaInfinityUI:
    """Interactive UI for Omega Infinity compression"""

    def __init__(self):
        self.compressor = OmegaInfinityCompressor(base_order=4, meta_levels=3, verbose=True)
        self.last_compressed = None
        self.last_decompressed = None
        self.setup_ui()

    def setup_ui(self):
        """Setup the interactive UI"""
        # Title
        self.title = HTML("""
        <div style="text-align: center; padding: 20px; background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); border-radius: 10px; margin-bottom: 20px;">
            <h1 style="color: #e94560; margin: 0;">🌌 OMEGA INFINITY v4.0</h1>
            <p style="color: #a0a0a0; margin: 10px 0 0 0;">Meta-Recursive Markov Chain Compression</p>
            <p style="color: #606060; font-size: 12px;">f(m) = m applied recursively</p>
        </div>
        """)

        # Mode selector
        self.mode_selector = widgets.RadioButtons(
            options=['📝 Compress Text', '📁 Compress File', '🔓 Decompress Text', '📂 Decompress File'],
            value='📝 Compress Text',
            description='Mode:',
            style={'description_width': 'initial'}
        )
        self.mode_selector.observe(self.on_mode_change, names='value')

        # Parameters
        self.order_slider = widgets.IntSlider(
            value=4, min=1, max=8, step=1,
            description='Markov Order:',
            style={'description_width': 'initial'}
        )

        self.levels_slider = widgets.IntSlider(
            value=3, min=1, max=5, step=1,
            description='Meta-Levels:',
            style={'description_width': 'initial'}
        )

        # Text input
        self.text_input = widgets.Textarea(
            placeholder='Enter text to compress here...',
            layout=widgets.Layout(width='100%', height='200px')
        )

        # Compressed input (for decompression)
        self.compressed_input = widgets.Textarea(
            placeholder='Paste compressed data (base64) here...',
            layout=widgets.Layout(width='100%', height='200px')
        )

        # File upload
        self.file_upload = widgets.FileUpload(
            accept='',
            multiple=False,
            description='Upload File'
        )

        # Action button
        self.action_button = widgets.Button(
            description='🚀 Compress',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px')
        )
        self.action_button.on_click(self.on_action)

        # Download button
        self.download_button = widgets.Button(
            description='💾 Download Result',
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px'),
            disabled=True
        )
        self.download_button.on_click(self.on_download)

        # Output area
        self.output = widgets.Output()

        # Result display
        self.result_area = widgets.Textarea(
            placeholder='Result will appear here...',
            layout=widgets.Layout(width='100%', height='200px'),
            disabled=True
        )

        # Stats display
        self.stats_html = widgets.HTML(value='')

        # Containers
        self.input_container = widgets.VBox([self.text_input])
        self.params_box = widgets.HBox([self.order_slider, self.levels_slider])
        self.buttons_box = widgets.HBox([self.action_button, self.download_button])

    def on_mode_change(self, change):
        """Handle mode change"""
        mode = change['new']

        if mode == '📝 Compress Text':
            self.input_container.children = [self.text_input]
            self.action_button.description = '🚀 Compress'
            self.action_button.button_style = 'success'
            self.text_input.placeholder = 'Enter text to compress here...'

        elif mode == '📁 Compress File':
            self.input_container.children = [self.file_upload]
            self.action_button.description = '🚀 Compress File'
            self.action_button.button_style = 'success'

        elif mode == '🔓 Decompress Text':
            self.input_container.children = [self.compressed_input]
            self.action_button.description = '🔓 Decompress'
            self.action_button.button_style = 'warning'
            self.compressed_input.placeholder = 'Paste compressed data (base64) here...'

        elif mode == '📂 Decompress File':
            self.input_container.children = [self.file_upload]
            self.action_button.description = '🔓 Decompress File'
            self.action_button.button_style = 'warning'

        self.download_button.disabled = True
        self.result_area.value = ''
        self.stats_html.value = ''

    def on_action(self, button):
        """Handle action button click"""
        mode = self.mode_selector.value

        self.compressor = OmegaInfinityCompressor(
            base_order=self.order_slider.value,
            meta_levels=self.levels_slider.value,
            verbose=True
        )

        with self.output:
            clear_output()

            try:
                if mode == '📝 Compress Text':
                    self.compress_text()
                elif mode == '📁 Compress File':
                    self.compress_file()
                elif mode == '🔓 Decompress Text':
                    self.decompress_text()
                elif mode == '📂 Decompress File':
                    self.decompress_file()

            except Exception as e:
                print(f"❌ Error: {str(e)}")
                import traceback
                traceback.print_exc()

    def compress_text(self):
        """Compress text input"""
        text = self.text_input.value

        if not text:
            print("⚠️ Please enter some text to compress!")
            return

        # Compress
        compressed = self.compressor.compress(text)
        self.last_compressed = compressed

        # Convert to base64 for display
        compressed_b64 = base64.b64encode(compressed).decode('ascii')
        self.result_area.value = compressed_b64

        # Stats
        original_size = len(text.encode('utf-8'))
        compressed_size = len(compressed)
        ratio = (1 - compressed_size / original_size) * 100

        self.stats_html.value = f"""
        <div style="background: #1a1a2e; padding: 15px; border-radius: 10px; margin-top: 10px;">
            <h3 style="color: #e94560; margin: 0 0 10px 0;">📊 Compression Stats</h3>
            <table style="color: white; width: 100%;">
                <tr><td>📁 Original:</td><td><b>{original_size:,} bytes</b></td></tr>
                <tr><td>📦 Compressed:</td><td><b>{compressed_size:,} bytes</b></td></tr>
                <tr><td>📉 Ratio:</td><td><b>{ratio:.2f}%</b></td></tr>
                <tr><td>🔢 Factor:</td><td><b>{original_size/compressed_size:.2f}x</b></td></tr>
            </table>
        </div>
        """

        self.download_button.disabled = False
        print("\n✅ Compression complete! You can copy the result or download it.")

    def compress_file(self):
        """Compress uploaded file"""
        if not self.file_upload.value:
            print("⚠️ Please upload a file first!")
            return

        # Get file data
        uploaded = list(self.file_upload.value.values())[0]
        filename = list(self.file_upload.value.keys())[0]
        content = uploaded['content']

        print(f"📁 Processing file: {filename}")
        print(f"📊 File size: {len(content):,} bytes\n")

        # Compress
        compressed = self.compressor.compress(content, is_binary=True)
        self.last_compressed = compressed
        self.last_filename = filename + '.omega'

        # Stats
        original_size = len(content)
        compressed_size = len(compressed)
        ratio = (1 - compressed_size / original_size) * 100

        self.stats_html.value = f"""
        <div style="background: #1a1a2e; padding: 15px; border-radius: 10px; margin-top: 10px;">
            <h3 style="color: #e94560; margin: 0 0 10px 0;">📊 Compression Stats</h3>
            <table style="color: white; width: 100%;">
                <tr><td>📁 Original:</td><td><b>{original_size:,} bytes</b></td></tr>
                <tr><td>📦 Compressed:</td><td><b>{compressed_size:,} bytes</b></td></tr>
                <tr><td>📉 Ratio:</td><td><b>{ratio:.2f}%</b></td></tr>
                <tr><td>🔢 Factor:</td><td><b>{original_size/compressed_size:.2f}x</b></td></tr>
                <tr><td>📄 Output:</td><td><b>{self.last_filename}</b></td></tr>
            </table>
        </div>
        """

        self.result_area.value = f"[Binary compressed data - {compressed_size:,} bytes]\nClick 'Download Result' to save."
        self.download_button.disabled = False
        print("\n✅ Compression complete! Click 'Download Result' to save the file.")

    def decompress_text(self):
        """Decompress text input"""
        compressed_b64 = self.compressed_input.value.strip()

        if not compressed_b64:
            print("⚠️ Please paste compressed data!")
            return

        # Decode base64
        try:
            compressed = base64.b64decode(compressed_b64)
        except:
            print("❌ Invalid base64 data!")
            return

        # Decompress
        decompressed = self.compressor.decompress(compressed)
        self.last_decompressed = decompressed

        if isinstance(decompressed, bytes):
            self.result_area.value = f"[Binary data - {len(decompressed):,} bytes]\nClick 'Download Result' to save."
        else:
            self.result_area.value = decompressed

        self.stats_html.value = f"""
        <div style="background: #1a1a2e; padding: 15px; border-radius: 10px; margin-top: 10px;">
            <h3 style="color: #4ecca3; margin: 0 0 10px 0;">✅ Decompression Complete</h3>
            <table style="color: white; width: 100%;">
                <tr><td>📦 Compressed:</td><td><b>{len(compressed):,} bytes</b></td></tr>
                <tr><td>📁 Decompressed:</td><td><b>{len(decompressed) if isinstance(decompressed, bytes) else len(decompressed.encode('utf-8')):,} bytes</b></td></tr>
            </table>
        </div>
        """

        self.download_button.disabled = False
        print("\n✅ Decompression complete!")

    def decompress_file(self):
        """Decompress uploaded file"""
        if not self.file_upload.value:
            print("⚠️ Please upload a .omega file first!")
            return

        # Get file data
        uploaded = list(self.file_upload.value.values())[0]
        filename = list(self.file_upload.value.keys())[0]
        content = uploaded['content']

        print(f"📁 Processing file: {filename}")
        print(f"📊 File size: {len(content):,} bytes\n")

        # Decompress
        decompressed = self.compressor.decompress(bytes(content))
        self.last_decompressed = decompressed

        # Determine output filename
        if filename.endswith('.omega'):
            self.last_filename = filename[:-6]
        else:
            self.last_filename = filename + '.decompressed'

        self.stats_html.value = f"""
        <div style="background: #1a1a2e; padding: 15px; border-radius: 10px; margin-top: 10px;">
            <h3 style="color: #4ecca3; margin: 0 0 10px 0;">✅ Decompression Complete</h3>
            <table style="color: white; width: 100%;">
                <tr><td>📦 Compressed:</td><td><b>{len(content):,} bytes</b></td></tr>
                <tr><td>📁 Decompressed:</td><td><b>{len(decompressed) if isinstance(decompressed, bytes) else len(decompressed.encode('utf-8')):,} bytes</b></td></tr>
                <tr><td>📄 Output:</td><td><b>{self.last_filename}</b></td></tr>
            </table>
        </div>
        """

        if isinstance(decompressed, bytes):
            self.result_area.value = f"[Binary data - {len(decompressed):,} bytes]\nClick 'Download Result' to save."
        else:
            self.result_area.value = decompressed[:5000] + ('...' if len(decompressed) > 5000 else '')

        self.download_button.disabled = False
        print("\n✅ Decompression complete! Click 'Download Result' to save.")

    def on_download(self, button):
        """Handle download button click"""
        mode = self.mode_selector.value

        if 'Compress' in mode:
            if self.last_compressed:
                filename = getattr(self, 'last_filename', 'compressed.omega')
                with open(filename, 'wb') as f:
                    f.write(self.last_compressed)
                files.download(filename)
        else:
            if self.last_decompressed:
                filename = getattr(self, 'last_filename', 'decompressed.txt')
                mode = 'wb' if isinstance(self.last_decompressed, bytes) else 'w'
                with open(filename, mode) as f:
                    f.write(self.last_decompressed)
                files.download(filename)

    def display(self):
        """Display the UI"""
        display(self.title)
        display(widgets.HTML("<h3>⚙️ Settings</h3>"))
        display(self.mode_selector)
        display(self.params_box)
        display(widgets.HTML("<h3>📥 Input</h3>"))
        display(self.input_container)
        display(self.buttons_box)
        display(widgets.HTML("<h3>📤 Output</h3>"))
        display(self.output)
        display(self.result_area)
        display(self.stats_html)


# ============================================================
# SIMPLE COMMAND LINE INTERFACE (Alternative)
# ============================================================

def compress_text(text, order=4, levels=3):
    """
    Simple function to compress text

    Args:
        text: String to compress
        order: Markov order (1-8)
        levels: Meta-levels (1-5)

    Returns:
        Compressed bytes
    """
    compressor = OmegaInfinityCompressor(base_order=order, meta_levels=levels)
    return compressor.compress(text)

def decompress_data(compressed):
    """
    Simple function to decompress data

    Args:
        compressed: Compressed bytes

    Returns:
        Original data
    """
    compressor = OmegaInfinityCompressor()
    return compressor.decompress(compressed)

def compress_to_file(text, filename="output.omega", order=4, levels=3):
    """Compress text and save to file"""
    compressed = compress_text(text, order, levels)
    with open(filename, 'wb') as f:
        f.write(compressed)
    files.download(filename)
    return filename

def decompress_from_file(filename):
    """Load and decompress a file"""
    with open(filename, 'rb') as f:
        compressed = f.read()
    return decompress_data(compressed)


# ============================================================
# MAIN - LAUNCH UI
# ============================================================

print("""
╔══════════════════════════════════════════════════════════════╗
║                                                              ║
║     🌌 OMEGA INFINITY v4.0 - Meta-Recursive Compression 🌌   ║
║                                                              ║
║     • Compress text or files                                 ║
║     • Decompress .omega files                                ║
║     • Uses adaptive Markov chains                            ║
║     • Recursive meta-compression: f(f(f(m)))                 ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
""")

# Launch interactive UI
ui = OmegaInfinityUI()
ui.display()

print("\n" + "="*60)
print("💡 QUICK USAGE (Alternative to UI):")
print("="*60)
print("""
# Compress text:
compressed = compress_text("Your text here...")

# Decompress:
original = decompress_data(compressed)

# Save to file:
compress_to_file("Your text...", "myfile.omega")

# Load from file:
uploaded = files.upload()  # Upload .omega file
original = decompress_from_file("filename.omega")
""")


╔══════════════════════════════════════════════════════════════╗
║                                                              ║
║     🌌 OMEGA INFINITY v4.0 - Meta-Recursive Compression 🌌   ║
║                                                              ║
║     • Compress text or files                                 ║
║     • Decompress .omega files                                ║
║     • Uses adaptive Markov chains                            ║
║     • Recursive meta-compression: f(f(f(m)))                 ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝



HTML(value='<h3>⚙️ Settings</h3>')

RadioButtons(description='Mode:', options=('📝 Compress Text', '📁 Compress File', '🔓 Decompress Text', '📂 Decom…

HTML(value='<h3>📥 Input</h3>')

HTML(value='<h3>📤 Output</h3>')

Output()

Textarea(value='', disabled=True, layout=Layout(height='200px', width='100%'), placeholder='Result will appear…

HTML(value='')


💡 QUICK USAGE (Alternative to UI):

# Compress text:
compressed = compress_text("Your text here...")

# Decompress:
original = decompress_data(compressed)

# Save to file:
compress_to_file("Your text...", "myfile.omega")

# Load from file:
uploaded = files.upload()  # Upload .omega file
original = decompress_from_file("filename.omega")

